<a href="https://colab.research.google.com/github/RebecaGis/Script-Gen-rico-ArcGIS-Pro-Notebook-Identifica-o-de-Tipo-de-Logradouro-Pr-ximo/blob/main/Script_Gen%C3%A9rico_ArcGIS_Pro_Notebook_%E2%80%93_Identifica%C3%A7%C3%A3o_de_Tipo_de_Logradouro_Pr%C3%B3ximo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy
import os
from ipywidgets import FileUpload, Text, VBox, Button, Label
from IPython.display import display

# Configurações do ambiente
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False

# Interface para entrada dos arquivos
print("📂 Selecione os shapefiles e informe o caminho de saída.")

# Entradas de texto para caminhos
logradouros_input = Text(description="Logradouros:")
individuos_input = Text(description="Indivíduos:")
saida_input = Text(description="Saída:")

# Botão de execução
executar_btn = Button(description="Executar", button_style='success')
output_label = Label()

# Função principal
def executar_script(b):
    logradouros_shp = logradouros_input.value
    individuos_shp = individuos_input.value
    output_individuos = saida_input.value

    if not (os.path.exists(logradouros_shp) and os.path.exists(individuos_shp)):
        output_label.value = "❌ Caminhos inválidos. Verifique os shapefiles."
        return

    try:
        arcpy.management.CopyFeatures(individuos_shp, output_individuos)

        campo_tipo_log = "TpLograd"
        try:
            arcpy.management.AddField(output_individuos, campo_tipo_log, "TEXT", field_length=50)
        except arcpy.ExecuteError:
            print(f"⚠️ Campo '{campo_tipo_log}' já existe.")

        logradouros_layer = "logradouros_layer"
        arcpy.management.MakeFeatureLayer(logradouros_shp, logradouros_layer)

        try:
            arcpy.management.AddSpatialIndex(logradouros_layer)
        except:
            pass

        sr = arcpy.Describe(output_individuos).spatialReference

        with arcpy.da.UpdateCursor(output_individuos, ["SHAPE@XY", campo_tipo_log]) as cursor:
            for i, row in enumerate(cursor, 1):
                x, y = row[0]
                ponto = arcpy.PointGeometry(arcpy.Point(x, y), sr)

                tipo_logradouro = None
                distancia_minima = float('inf')

                arcpy.management.SelectLayerByLocation(logradouros_layer, "WITHIN_A_DISTANCE", ponto, "50 METERS")

                count = int(arcpy.management.GetCount(logradouros_layer).getOutput(0))
                if count > 0:
                    with arcpy.da.SearchCursor(logradouros_layer, ["SHAPE@", "NM_TIP_LOG"]) as log_cursor:
                        for log in log_cursor:
                            distancia = ponto.distanceTo(log[0])
                            if distancia < distancia_minima:
                                distancia_minima = distancia
                                tipo_logradouro = log[1]

                if tipo_logradouro is None:
                    with arcpy.da.SearchCursor(logradouros_layer, ["SHAPE@", "NM_TIP_LOG"]) as log_cursor:
                        for log in log_cursor:
                            distancia = ponto.distanceTo(log[0])
                            if distancia < distancia_minima:
                                distancia_minima = distancia
                                tipo_logradouro = log[1]

                row[1] = tipo_logradouro if tipo_logradouro else "NÃO ENCONTRADO"
                cursor.updateRow(row)

                if i % 100 == 0:
                    print(f"✅ {i} pontos processados...")

        print("\n✅ Processo concluído com sucesso!")
        print(f"📁 Shapefile gerado em: {output_individuos}")

        print("\n📊 Resumo de tipos de logradouros encontrados:")
        tipo_counts = {}
        with arcpy.da.SearchCursor(output_individuos, [campo_tipo_log]) as cursor:
            for row in cursor:
                tipo = row[0] if row[0] else "NÃO IDENTIFICADO"
                tipo_counts[tipo] = tipo_counts.get(tipo, 0) + 1

        for tipo, count in sorted(tipo_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"{tipo}: {count} indivíduos")

        output_label.value = "✅ Script finalizado com sucesso!"

    except Exception as e:
        output_label.value = f"❌ Erro: {e}"

# Vincular botão
executar_btn.on_click(executar_script)

# Exibir interface
display(VBox([logradouros_input, individuos_input, saida_input, executar_btn, output_label]))
